In [22]:
import os 
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import re

In [ ]:
property = pd.read_csv('data/Market_value_usable_hedonic_data.csv')

In [42]:
property.columns

Index(['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0',
       'FullStreetNumber', 'RoadName', 'suburb_loc', 'YearSold', 'AgeAtSale',
       'GrossSalePrice', 'CapitalValue', 'LandValue', 'ImprovementsValue',
       'PriceValueRelationship', 'LandArea', 'TotalFloorArea', 'gd2000co',
       'gd2000_yco', 'cycleways_DIST', 'cycle_DENS', 'on_DIST', 'on_DENS',
       'Green_DIST', 'water_DIST', 'bus_DIST', 'Census_Pop', 'RnkIMDNoEm',
       'RnkIMDNoIn', 'RnkIMDNoCr', 'RnkIMDNoHo', 'RnkIMDNoHe', 'RnkIMDNoEd',
       'RnkIMDNoAc', 'DECILE_high', 'DECILE_prim', 'Median_Income', 'CBD_DIST',
       'Ratio', 'z_score'],
      dtype='object')

In [24]:
property_cleaned = property.loc[:,~property.columns.str.contains('^Unnamed')]

In [25]:
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:,.0f}'.format)

In [26]:
data_2024 = property_cleaned[property_cleaned['YearSold'] == 2022]

In [27]:
pp_suburb = data_2024.groupby('suburb_loc').agg(
  median_price = ('GrossSalePrice', 'median'),
  mean_price = ('GrossSalePrice', 'mean'),
  sales_count = ('GrossSalePrice', 'count')
).sort_values('median_price', ascending=False)

In [28]:
sa2 = gpd.read_file('data/chc-boundaries/statistical-area-2-2023-generalised.shp')
ta = gpd.read_file('data/chc-boundaries/territorial-authority-2021-generalised.gpkg')

In [29]:
pp_suburb_reset = pp_suburb.reset_index()

In [30]:
pp_suburb_reset.head()

,suburb_loc,median_price,mean_price,sales_count
0,Ouruhia,"1,272,000","1,272,000",2
1,Fendalton,"1,165,000","1,137,964",28
2,Bottle Lake,"1,153,000","1,153,000",2
3,Merivale,"1,040,000","1,060,383",47
4,Huntsbury,"985,000","983,712",31


In [31]:
ta_chc = ta[ta['TA2021_V1_00_NAME_ASCII'].str.lower().str.contains('christchurch')]

In [32]:
sa2_chc = sa2.clip(ta_chc)

In [33]:
pp_suburb_reset[pp_suburb_reset['suburb_loc'].str.lower().str.contains('martin')]

,suburb_loc,median_price,mean_price,sales_count
26,Saint Martins,"720,000","726,824",37


In [34]:
sa2_chc["suburb_loc"] = (
    sa2_chc["SA22023__2"]
      .str.replace(r"\s*\(Christchurch City\)", "", regex=True)
      .str.replace(r"\s+(East|West|North|South|Central)$", "", regex=True)
      .str.replace(r"-(West|East|North|South)$", "", regex=True)
)

In [35]:
sa2_chc[sa2_chc['suburb_loc'].str.lower().str.contains('martin')]

,SA22023_V1,SA22023__1,SA22023__2,LAND_AREA_,AREA_SQ_KM,Shape_Leng,geometry,suburb_loc
206,330800,St Martins,St Martins,1,1,"5,748","POLYGON ((1572257.48 5177758.528, 1572287.989 ...",St Martins


In [36]:
pp_suburb_reset['suburb_loc'] = (
  pp_suburb_reset['suburb_loc'].str.replace(r"^Saint\s+", "St ", regex=True)  
)

In [37]:
suburb_geom = sa2_chc.dissolve(by="suburb_loc", as_index=False)

In [38]:
suburb_prices = suburb_geom.merge(pp_suburb_reset, on="suburb_loc", how="left")

In [39]:
pp_suburb_reset[~pp_suburb_reset['suburb_loc'].isin(suburb_geom['suburb_loc'])]

,suburb_loc,median_price,mean_price,sales_count
0,Ouruhia,"1,272,000","1,272,000",2
2,Bottle Lake,"1,153,000","1,153,000",2
15,Cracroft,"835,750","796,417",6
16,Brooklands,"800,100","800,100",1
22,Spencerville,"740,000","693,750",8
52,North New Brighton,"575,000","575,000",1
68,Kainga,"424,500","467,375",8


In [40]:
suburb_prices.head()

,suburb_loc,geometry,SA22023_V1,SA22023__1,SA22023__2,LAND_AREA_,AREA_SQ_KM,Shape_Leng,median_price,mean_price,sales_count
0,Addington,"POLYGON ((1569941.614 5178866.399, 1569942.698...",327400,Addington East,Addington East,1,1,"3,923","590,000","566,838",96
1,Aidanfield,"POLYGON ((1565173.494 5177250.222, 1565177.286...",325400,Aidanfield,Aidanfield,2,2,"6,272","945,000","967,392",51
2,Akaroa,"POLYGON ((1597814.837 5150724.97, 1597855.848 ...",333500,Akaroa,Akaroa,2,2,"11,500",NaN,NaN,NaN
3,Akaroa Harbour,"POLYGON ((1597850.857 5159834.259, 1597829.823...",333300,Akaroa Harbour,Akaroa Harbour,162,162,"150,521",NaN,NaN,NaN
4,Aranui,"POLYGON ((1575895.095 5181699.091, 1575824.43 ...",328600,Aranui,Aranui,1,1,"4,670","460,500","448,646",54


In [41]:
m = suburb_prices.explore(
    column="median_price",
    cmap="Reds",                 # continuous gradient from low→high
    legend=True,
    tooltip=["suburb_loc", "median_price", "sales_count"],
    tiles="CartoDB positron",
    missing_kwds={
        "color": "lightgrey",
        "label": "No sales data",
        "hatch": "///",
    },
    style_kwds={
        "weight": 0.3,
        "opacity": 0.8,
        "color": "black",
    }
)

m


/Users/rujalshrestha/Projects/chc-property-prices/venv/lib/python3.13/site-packages/folium/features.py:1204: UserWarning: GeoJsonTooltip is not configured to render for GeoJson GeometryCollection geometries. Please consider reworking these features: [{'suburb_loc': 'Oceanic Canterbury Region', 'SA22023_V1': '363800', 'SA22023__1': 'Oceanic Canterbury Region', 'SA22023__2': 'Oceanic Canterbury Region', 'LAND_AREA_': 0.0, 'AREA_SQ_KM': 11441.92869308, 'Shape_Leng': 1176105.5925440695, 'median_price': None, 'mean_price': None, 'sales_count': None, '__folium_color': 'lightgrey'}, {'suburb_loc': 'Prebbleton', 'SA22023_V1': '335701', 'SA22023__1': 'Prebbleton North', 'SA22023__2': 'Prebbleton North', 'LAND_AREA_': 3.33201218, 'AREA_SQ_KM': 3.33201218, 'Shape_Leng': 7233.725192121311, 'median_price': None, 'mean_price': None, 'sales_count': None, '__folium_color': 'lightgrey'}, {'suburb_loc': 'Trents', 'SA22023_V1': '335600', 'SA22023__1': 'Trents', 'SA22023__2': 'Trents', 'LAND_AREA_': 50.01